<a href="https://colab.research.google.com/github/JamesKha/MovieRecommendation/blob/main/MiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from nltk.stem.porter import *
import string
from rake_nltk import Rake

In [2]:
linksDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/links.csv')
moviesDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/movies.csv')
ratingsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/ratings.csv')
tagsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/tags.csv')

#Inspecting the dataframes

In [3]:
linksDF.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
moviesDF.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratingsDF.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tagsDF.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


#Connecting the dataframes based on movie id number

##Title and genres added

In [7]:
userRatingsDataFrame = pd.merge(linksDF, ratingsDF, on='movieId' )
userRatingsDataFrame = userRatingsDataFrame.rename({'timestamp':'ratingTimestamp'},axis='columns')
userRatingsDataFrame = pd.merge(userRatingsDataFrame, moviesDF, on='movieId')
userRatingsDataFrame

,movieId,imdbId,tmdbId,userId,rating,ratingTimestamp,title,genres
0,1,114709,862.0,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,114709,862.0,5,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,114709,862.0,7,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,114709,862.0,15,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,114709,862.0,17,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...,...,...
100831,193581,5476944,432131.0,184,4.0,1537109082,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
100832,193583,5914996,445030.0,184,3.5,1537109545,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
100833,193585,6397426,479308.0,184,3.5,1537109805,Flint (2017),Drama
100834,193587,8391976,483455.0,184,3.5,1537110021,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


#Cleaning text tags

In [8]:
tagsDF['tag'].str.lower().value_counts()

in netflix queue               131
atmospheric                     41
superhero                       24
funny                           24
surreal                         24
                              ... 
may-december romance             1
political right versus left      1
brittany murphy                  1
venice                           1
stone age                        1
Name: tag, Length: 1475, dtype: int64

##Removing numbers from the tags

In [9]:
tagsDF['tag'] = tagsDF['tag'].str.replace(r'\d+', r' ')

In [10]:
tagsDF['tag'] = tagsDF['tag'].str.replace(r'[^\w\s]', r' ')

In [11]:
tagsDFCopy = tagsDF

In [12]:
tagsDFCopy

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


##Experiementing with stemmer

In [13]:
stemmer = PorterStemmer()

In [14]:
tagsDF['tag'] = tagsDF['tag'].apply(lambda row: " ".join([stemmer.stem(word) for word in row.split(" ")]))


In [15]:
tagsDF

,userId,movieId,tag,timestamp
0,2,60756,funni,1445714994
1,2,60756,highli quotabl,1445714996
2,2,60756,will ferrel,1445714992
3,2,89774,box stori,1445715207
4,2,89774,mma,1445715200
...,...,...,...,...
3678,606,7382,for kati,1171234019
3679,606,7936,auster,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodsh,1493843978


##Experiementing ratings with string changes

In [16]:
ratingsTagsDf = pd.merge(ratingsDF, tagsDF, on='movieId')
tagVSRatings = pd.DataFrame(ratingsTagsDf[['tag','rating']].groupby('tag')['rating'].value_counts(sort=False))
tagVSRatings.rename({'rating':'count'},axis='columns')


count
tag             rating       
  like          1.0         1
                2.0         1
                3.0         4
                3.5         4
                4.0         9
...                       ...
zooey deschanel 3.0         2
                3.5         6
                4.0        15
                4.5         6
                5.0         5

[10308 rows x 1 columns]

###Number of unique tags

In [17]:
print('Number of unique tags: %i.'% len(ratingsTagsDf.tag.unique()))

Number of unique tags: 1420.


In [18]:
generalCountofTags = pd.DataFrame(ratingsTagsDf.tag.value_counts())
generalCountofTags.reset_index(inplace=True)
generalCountofTags

,index,tag
0,psycholog,3089
1,sci fi,2559
2,atmospher,2534
3,thought provok,2491
4,twist end,2440
...,...,...
1415,big wave,1
1416,system holism,1
1417,human right,1
1418,van gogh,1


##Affiliation of tags to the movies

In [19]:
tagsAndGenresDataFrame = pd.merge(tagsDF, moviesDF, on='movieId')[['title', 'tag','genres']]

In [20]:
titlesAndTags = pd.DataFrame(tagsAndGenresDataFrame.groupby(['title']).tag.unique())
titlesAndTags = titlesAndTags.reset_index()
titlesAndTags

,title,tag
0,(500) Days of Summer (2009),"[artist, funni, humor, inspir, intellig, quirk..."
1,...And Justice for All (1979),[lawyer]
2,10 Cloverfield Lane (2016),"[creepi, suspens]"
3,10 Things I Hate About You (1999),[shakespear sort of]
4,101 Dalmatians (1996),"[dog, remak]"
...,...,...
1567,Zero Dark Thirty (2012),"[afghanistan, american propaganda, assassin, m..."
1568,Zombieland (2009),"[bill murray, dark comedi, emma stone, funni, ..."
1569,Zoolander (2001),"[ben stiller, comedi, david bowi, goofi, mindl..."
1570,Zulu (1964),[africa]


In [21]:
genresAndTags = pd.DataFrame(tagsAndGenresDataFrame.groupby(['genres']).tag.unique())
genresAndTags = genresAndTags.reset_index()
genresAndTags

,genres,tag
0,(no genres listed),"[quirki, sweet, underst]"
1,Action|Adventure,"[adventur, archaeolog, indiana jone, steven sp..."
2,Action|Adventure|Animation|Children|Comedy,"[disney, superhero, anim, samuel L jackson]"
3,Action|Adventure|Animation|Children|Comedy|Fan...,"[cheeki, clever, color, feel good, fun, imagin..."
4,Action|Adventure|Animation|Children|Comedy|Sci-Fi,[someth for everyon in thi one saw it witho...
...,...,...
367,Sci-Fi,"[beauti visual, cerebr, cinematographi, good c..."
368,Sci-Fi|IMAX,"[black hole, sci fi, time travel, christoph no..."
369,Sci-Fi|Thriller,[good]
370,Thriller,"[horror, juliett lewi, martin scorses, robert ..."


##Experimenting with the NLTK_RAKE within the context of data cleaning 

In [22]:
tagsDFCopy

,userId,movieId,tag,timestamp
0,2,60756,funni,1445714994
1,2,60756,highli quotabl,1445714996
2,2,60756,will ferrel,1445714992
3,2,89774,box stori,1445715207
4,2,89774,mma,1445715200
...,...,...,...,...
3678,606,7382,for kati,1171234019
3679,606,7936,auster,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodsh,1493843978


In [23]:
key_words_keys = []
for index, row in tagsDFCopy.iterrows(): 
  tags = row['tag']
  r = Rake()

  r.extract_keywords_from_text(tags)

  key_words_dict_scores = r.get_word_degrees()

  key_words_keys.append(list(key_words_dict_scores.keys()))

tagsDFCopy['Key_words'] = key_words_keys


tagsDFCopy.drop(columns = ['tag'], inplace=True)

In [24]:
tagsDFCopy

,userId,movieId,timestamp,Key_words
0,2,60756,1445714994,[funni]
1,2,60756,1445714996,"[highli, quotabl]"
2,2,60756,1445714992,[ferrel]
3,2,89774,1445715207,"[box, stori]"
4,2,89774,1445715200,[mma]
...,...,...,...,...
3678,606,7382,1171234019,[kati]
3679,606,7936,1173392334,[auster]
3680,610,3265,1493843984,"[gun, fu]"
3681,610,3265,1493843978,"[heroic, bloodsh]"
